# Análise dos resultados da avaliação de descrições de alinhamentos
---
São Carlos, 16/12/2019

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from app.eval_module.desc_models import DescEval

def print_metrics(df):
    """

    :param df: pandas data-frame 
    :return: 
    """
    total = df.values.sum()
    print(f"Total:\t\t\t{df.values.sum()}")
    print(f"Valid:\t\t\t{df.loc[:, 'Correto':'Incorreto'].values.sum()} |\t{df.loc[:, 'Correto':'Incorreto'].values.sum()*100./total:.2f} %")
    print(f"Correto:\t\t{df.loc[:, 'Correto'].values.sum()} |\t{df.loc[:, 'Correto'].values.sum() * 100./total:.2f} %")
    print(f"Parcialmente correto:\t{df.loc[:, 'Parcialmente correto'].values.sum()} |\t{df.loc[:, 'Parcialmente correto'].values.sum() * 100./total:.2f} %")
    print(f"Incorreto:\t\t{df.loc[:, 'Incorreto'].values.sum()} |\t{df.loc[:, 'Incorreto'].values.sum() * 100./total:.2f} %")
    print(f"Inválido:\t\t{df.loc[:, 'Inválido'].values.sum()} |\t{df.loc[:, 'Inválido'].values.sum() * 100./total:.2f} %")
    print()
    print('Valid descriptions != alignment')
    df1: pd.DataFrame = df.loc[:, 'Correto':'Incorreto']
    # df1.loc[:, 'Correto'].sum()
    total = df1.values.sum() - df1.loc['Igual'].sum()
    Correto = df1.loc[:, 'Correto'].sum() - df1.loc['Igual', 'Correto'].sum()
    p_Correto = df1.loc[:, 'Parcialmente correto'].sum() - df1.loc['Igual', 'Parcialmente correto'].sum()
    Incorreto = df1.loc[:, 'Incorreto'].sum() - df1.loc['Igual', 'Incorreto'].sum()
    print(f"{Correto}\tValid NE. Correto:\t\t{Correto * 100./total:.2f} %")
    print(f"{p_Correto}\tValid NE. Parcialmente correto:\t{p_Correto * 100./total:.2f} %")
    print(f"{Incorreto}\tValid NE. Incorreto:\t\t{Incorreto * 100./total:.2f} %")
    print()
    print('Valid descriptions != alignment')
    df2 = df
    total = df2.values.sum()
    Melhor = df2.loc['Melhor'].sum()
    Igual = df2.loc['Igual'].sum()
    Pior = df2.loc['Pior'].sum()
    print(f"{Melhor}\tMelhor:\t{Melhor * 100./total:.2f} %")
    print(f"{Igual}\tIgual:\t{Igual * 100./total:.2f} %")
    print(f"{Pior}\tPior:\t{Pior * 100./total:.2f} %")
    print()
    print(df.to_latex())

In [ ]:
def df_int_to_perc(df):
    """
    :type df pd.DataFrame
    """
    total = df.values.sum()
    new_df = pd.DataFrame(
        [[f"{row[c_name] * 100./total:.2f} %" for c_name in column_names] for index, row in df.iterrows()]
        , columns=column_names, index=index_names
    )
    print(new_df.to_latex())
    return new_df


## 1. Análise geral

In [19]:
possible_approval = [2, 1, 0, -1]
possible_comparing = [2, 1, 0]

index_names = [DescEval.COMPARE_BASELINE_TABLE[pc] for pc in possible_comparing]
column_names = [DescEval.APPROVAL_TABLE[pa] for pa in possible_approval]

df = pd.DataFrame([
    [
        len([de for de in DescEval.query.filter_by(approval=pa, compare_baseline=pc)]) for pa in possible_approval
    ] for pc in possible_comparing
], columns=column_names,
index=index_names)

print_metrics(df)
df_int_to_perc(df)

Total:			596
Valid:			318 |	53.36 %
Correto:		111 |	18.62 %
Parcialmente correto:	49 |	8.22 %
Incorreto:		158 |	26.51 %
Inválido:		278 |	46.64 %

Valid descriptions != alignment
59	Valid NE. Correto:		22.61 %
44	Valid NE. Parcialmente correto:	16.86 %
158	Valid NE. Incorreto:		60.54 %

Valid descriptions != alignment
202	Melhor:	33.89 %
111	Igual:	18.62 %
283	Pior:	47.48 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &       59 &                    44 &          0 &        99 \\
Igual  &       52 &                     5 &          0 &        54 \\
Pior   &        0 &                     0 &        158 &       125 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} & Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &  9.90 \% &               7.38 \% &    0.00 \% &  16.61 \% \\
Igual  &  8.72 \% &               0.84 \% &    0.00 \% &   9.06 \% \\
Pior   &  0.00 \% &               0.

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,9.90 %,7.38 %,0.00 %,16.61 %
Igual,8.72 %,0.84 %,0.00 %,9.06 %
Pior,0.00 %,0.00 %,26.51 %,20.97 %


## 2. Análise por modelo

In [11]:
from app.desc_module.models import Description
df_modelos = {}

for key, value in Description.METHOD_TABLE.items():
    df_modelos[value] = pd.DataFrame([
        [
            len([de for de in DescEval.query.filter_by(approval=pa, compare_baseline=pc) if de.get_method() == value]) for pa in possible_approval
        ] for pc in possible_comparing
    ], columns=column_names,
    index=index_names)

### 2.1. Model1

In [12]:
print_metrics(df_modelos['Model1'])
df_int_to_perc(df_modelos['Model1'])

Total:			98
Valid:			54 |	55.10 %
Correto:		14 |	14.29 %
Parcialmente correto:	9 |	9.18 %
Incorreto:		31 |	31.63 %
Inválido:		44 |	44.90 %

Valid descriptions != alignment
13	Valid NE. Correto:		25.00 %
8	Valid NE. Parcialmente correto:	15.38 %
31	Valid NE. Incorreto:		59.62 %

Valid descriptions != alignment
35	Melhor:	35.71 %
5	Igual:	5.10 %
58	Pior:	59.18 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &       13 &                     8 &          0 &        14 \\
Igual  &        1 &                     1 &          0 &         3 \\
Pior   &        0 &                     0 &         31 &        27 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} &  Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &  13.27 \% &               8.16 \% &    0.00 \% &  14.29 \% \\
Igual  &   1.02 \% &               1.02 \% &    0.00 \% &   3.06 \% \\
Pior   &   0.00 \% &               0.00 \% &  

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,13.27 %,8.16 %,0.00 %,14.29 %
Igual,1.02 %,1.02 %,0.00 %,3.06 %
Pior,0.00 %,0.00 %,31.63 %,27.55 %


### 2.2. Model2

In [13]:
print_metrics(df_modelos['Model2'])
df_int_to_perc(df_modelos['Model2'])

Total:			98
Valid:			52 |	53.06 %
Correto:		27 |	27.55 %
Parcialmente correto:	6 |	6.12 %
Incorreto:		19 |	19.39 %
Inválido:		46 |	46.94 %

Valid descriptions != alignment
10	Valid NE. Correto:		28.57 %
6	Valid NE. Parcialmente correto:	17.14 %
19	Valid NE. Incorreto:		54.29 %

Valid descriptions != alignment
36	Melhor:	36.73 %
32	Igual:	32.65 %
30	Pior:	30.61 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &       10 &                     6 &          0 &        20 \\
Igual  &       17 &                     0 &          0 &        15 \\
Pior   &        0 &                     0 &         19 &        11 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} &  Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &  10.20 \% &               6.12 \% &    0.00 \% &  20.41 \% \\
Igual  &  17.35 \% &               0.00 \% &    0.00 \% &  15.31 \% \\
Pior   &   0.00 \% &               0.00 \% &

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,10.20 %,6.12 %,0.00 %,20.41 %
Igual,17.35 %,0.00 %,0.00 %,15.31 %
Pior,0.00 %,0.00 %,19.39 %,11.22 %


### 2.3. Model3

In [14]:
print_metrics(df_modelos['Model3'])
df_int_to_perc(df_modelos['Model3'])

Total:			100
Valid:			53 |	53.00 %
Correto:		10 |	10.00 %
Parcialmente correto:	10 |	10.00 %
Incorreto:		33 |	33.00 %
Inválido:		47 |	47.00 %

Valid descriptions != alignment
9	Valid NE. Correto:		17.65 %
9	Valid NE. Parcialmente correto:	17.65 %
33	Valid NE. Incorreto:		64.71 %

Valid descriptions != alignment
33	Melhor:	33.00 %
5	Igual:	5.00 %
62	Pior:	62.00 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &        9 &                     9 &          0 &        15 \\
Igual  &        1 &                     1 &          0 &         3 \\
Pior   &        0 &                     0 &         33 &        29 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} & Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &  9.00 \% &               9.00 \% &    0.00 \% &  15.00 \% \\
Igual  &  1.00 \% &               1.00 \% &    0.00 \% &   3.00 \% \\
Pior   &  0.00 \% &               0.00 \% &   3

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,9.00 %,9.00 %,0.00 %,15.00 %
Igual,1.00 %,1.00 %,0.00 %,3.00 %
Pior,0.00 %,0.00 %,33.00 %,29.00 %


### 2.4. Model4

In [15]:
print_metrics(df_modelos['Model4'])
df_int_to_perc(df_modelos['Model4'])

Total:			100
Valid:			53 |	53.00 %
Correto:		25 |	25.00 %
Parcialmente correto:	7 |	7.00 %
Incorreto:		21 |	21.00 %
Inválido:		47 |	47.00 %

Valid descriptions != alignment
9	Valid NE. Correto:		25.00 %
6	Valid NE. Parcialmente correto:	16.67 %
21	Valid NE. Incorreto:		58.33 %

Valid descriptions != alignment
33	Melhor:	33.00 %
32	Igual:	32.00 %
35	Pior:	35.00 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &        9 &                     6 &          0 &        18 \\
Igual  &       16 &                     1 &          0 &        15 \\
Pior   &        0 &                     0 &         21 &        14 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} &  Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &   9.00 \% &               6.00 \% &    0.00 \% &  18.00 \% \\
Igual  &  16.00 \% &               1.00 \% &    0.00 \% &  15.00 \% \\
Pior   &   0.00 \% &               0.00 \% &

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,9.00 %,6.00 %,0.00 %,18.00 %
Igual,16.00 %,1.00 %,0.00 %,15.00 %
Pior,0.00 %,0.00 %,21.00 %,14.00 %


### 2.5. Model5

In [16]:
print_metrics(df_modelos['Model5'])
df_int_to_perc(df_modelos['Model5'])

Total:			100
Valid:			53 |	53.00 %
Correto:		10 |	10.00 %
Parcialmente correto:	10 |	10.00 %
Incorreto:		33 |	33.00 %
Inválido:		47 |	47.00 %

Valid descriptions != alignment
9	Valid NE. Correto:		17.65 %
9	Valid NE. Parcialmente correto:	17.65 %
33	Valid NE. Incorreto:		64.71 %

Valid descriptions != alignment
33	Melhor:	33.00 %
5	Igual:	5.00 %
62	Pior:	62.00 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &        9 &                     9 &          0 &        15 \\
Igual  &        1 &                     1 &          0 &         3 \\
Pior   &        0 &                     0 &         33 &        29 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} & Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &  9.00 \% &               9.00 \% &    0.00 \% &  15.00 \% \\
Igual  &  1.00 \% &               1.00 \% &    0.00 \% &   3.00 \% \\
Pior   &  0.00 \% &               0.00 \% &   3

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,9.00 %,9.00 %,0.00 %,15.00 %
Igual,1.00 %,1.00 %,0.00 %,3.00 %
Pior,0.00 %,0.00 %,33.00 %,29.00 %


### 2.6. Model6

In [17]:
print_metrics(df_modelos['Model6'])
df_int_to_perc(df_modelos['Model6'])


Total:			100
Valid:			53 |	53.00 %
Correto:		25 |	25.00 %
Parcialmente correto:	7 |	7.00 %
Incorreto:		21 |	21.00 %
Inválido:		47 |	47.00 %

Valid descriptions != alignment
9	Valid NE. Correto:		25.00 %
6	Valid NE. Parcialmente correto:	16.67 %
21	Valid NE. Incorreto:		58.33 %

Valid descriptions != alignment
32	Melhor:	32.00 %
32	Igual:	32.00 %
36	Pior:	36.00 %

\begin{tabular}{lrrrr}
\toprule
{} &  Correto &  Parcialmente correto &  Incorreto &  Inválido \\
\midrule
Melhor &        9 &                     6 &          0 &        17 \\
Igual  &       16 &                     1 &          0 &        15 \\
Pior   &        0 &                     0 &         21 &        15 \\
\bottomrule
\end{tabular}

\begin{tabular}{lllll}
\toprule
{} &  Correto & Parcialmente correto & Incorreto & Inválido \\
\midrule
Melhor &   9.00 \% &               6.00 \% &    0.00 \% &  17.00 \% \\
Igual  &  16.00 \% &               1.00 \% &    0.00 \% &  15.00 \% \\
Pior   &   0.00 \% &               0.00 \% &

,Correto,Parcialmente correto,Incorreto,Inválido
Melhor,9.00 %,6.00 %,0.00 %,17.00 %
Igual,16.00 %,1.00 %,0.00 %,15.00 %
Pior,0.00 %,0.00 %,21.00 %,15.00 %


In [18]:
df_comparacao = pd.DataFrame([
    [
        f"{(df.loc[:, 'Correto':'Incorreto'].loc[:, 'Correto'].sum() - df.loc[:, 'Correto':'Incorreto'].loc['Igual', 'Correto'].sum()) * 100/(df.loc[:, 'Correto':'Incorreto'].values.sum() - df.loc[:, 'Correto':'Incorreto'].loc['Igual'].sum()):.2f} %",
        f"{(df.loc[:, 'Correto':'Incorreto'].loc[:, 'Parcialmente correto'].sum() - df.loc[:, 'Correto':'Incorreto'].loc['Igual', 'Parcialmente correto'].sum()) * 100/(df.loc[:, 'Correto':'Incorreto'].values.sum() - df.loc[:, 'Correto':'Incorreto'].loc['Igual'].sum()):.2f} %",
        f"{(df.loc[:, 'Correto':'Incorreto'].loc[:, 'Incorreto'].sum() - df.loc[:, 'Correto':'Incorreto'].loc['Igual', 'Incorreto'].sum()) * 100/(df.loc[:, 'Correto':'Incorreto'].values.sum() - df.loc[:, 'Correto':'Incorreto'].loc['Igual'].sum()):.2f} %",
        f"{df.loc['Melhor'].sum() * 100./df.values.sum() :.2f} %",
        f"{df.loc['Igual'].sum() * 100./df.values.sum():.2f} %",
        f"{df.loc['Pior'].sum() * 100./df.values.sum():.2f} %"
    ] for _, df in df_modelos.items()
], columns=['Correto', 'Parcialmente correto', 'Incorreto', 'Melhores', 'Iguais', 'Piores'],
index=[key for key, _ in df_modelos.items()])

print(df_comparacao.to_latex())

df_comparacao

\begin{tabular}{lllllll}
\toprule
{} &  Correto & Parcialmente correto & Incorreto & Melhores &   Iguais &   Piores \\
\midrule
Model1 &  25.00 \% &              15.38 \% &   59.62 \% &  35.71 \% &   5.10 \% &  59.18 \% \\
Model2 &  28.57 \% &              17.14 \% &   54.29 \% &  36.73 \% &  32.65 \% &  30.61 \% \\
Model3 &  17.65 \% &              17.65 \% &   64.71 \% &  33.00 \% &   5.00 \% &  62.00 \% \\
Model4 &  25.00 \% &              16.67 \% &   58.33 \% &  33.00 \% &  32.00 \% &  35.00 \% \\
Model5 &  17.65 \% &              17.65 \% &   64.71 \% &  33.00 \% &   5.00 \% &  62.00 \% \\
Model6 &  25.00 \% &              16.67 \% &   58.33 \% &  32.00 \% &  32.00 \% &  36.00 \% \\
\bottomrule
\end{tabular}



,Correto,Parcialmente correto,Incorreto,Melhores,Iguais,Piores
Model1,25.00 %,15.38 %,59.62 %,35.71 %,5.10 %,59.18 %
Model2,28.57 %,17.14 %,54.29 %,36.73 %,32.65 %,30.61 %
Model3,17.65 %,17.65 %,64.71 %,33.00 %,5.00 %,62.00 %
Model4,25.00 %,16.67 %,58.33 %,33.00 %,32.00 %,35.00 %
Model5,17.65 %,17.65 %,64.71 %,33.00 %,5.00 %,62.00 %
Model6,25.00 %,16.67 %,58.33 %,32.00 %,32.00 %,36.00 %
